In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44879
2,Huelva,Confirmados PDIA 14 días,537
3,Huelva,Tasa PDIA 14 días,"104,64368532844867"
4,Huelva,Confirmados PDIA 7 días,218
5,Huelva,Tasa PDIA 7 dias,"42,481049164994054"
6,Huelva,Total Confirmados,45078
7,Huelva,Curados,41520
8,Huelva,Fallecidos,410


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44879.0


/tmp/ipykernel_45163/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12421.0


/tmp/ipykernel_45163/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_45163/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_45163/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_45163/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3995 personas en los últimos 7 días 

Un positivo PCR cada 1410 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44879.0,218.0,537.0,513170.0,42.481049,104.643685,34.0
Huelva-Costa,26561.0,167.0,368.0,289548.0,57.676102,127.094644,25.0
Huelva (capital),12421.0,36.0,102.0,143837.0,25.028331,70.913604,9.0
Ayamonte,2027.0,27.0,44.0,21104.0,127.937832,208.491281,5.0
Sierra de Huelva-Andévalo Central,3971.0,21.0,50.0,67391.0,31.161431,74.193883,5.0
Condado-Campiña,13926.0,27.0,109.0,156231.0,17.282102,69.768484,4.0
Isla Cristina,3093.0,28.0,59.0,21393.0,130.883934,275.791147,4.0
Cartaya,1927.0,47.0,77.0,20083.0,234.028781,383.408853,3.0
Lepe,2978.0,16.0,37.0,27880.0,57.388809,132.711621,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Nava (La),9.0,2.0,2.0,258.0,775.193798,775.193798,NaN
Cañaveral de León,32.0,1.0,3.0,396.0,252.525253,757.575758,0.0
Higuera de la Sierra,64.0,2.0,6.0,1291.0,154.918668,464.756003,0.0
Villablanca,284.0,3.0,12.0,2885.0,103.986135,415.944541,0.0
Aroche,127.0,7.0,12.0,3054.0,229.207597,392.927308,0.0
Cartaya,1927.0,47.0,77.0,20083.0,234.028781,383.408853,3.0
Puebla de Guzmán,174.0,1.0,9.0,3092.0,32.341527,291.073739,0.0
Isla Cristina,3093.0,28.0,59.0,21393.0,130.883934,275.791147,4.0
Ayamonte,2027.0,27.0,44.0,21104.0,127.937832,208.491281,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Puebla de Guzmán,174.0,1.0,9.0,3092.0,32.341527,291.073739,0.0,0.111111
Moguer,1929.0,4.0,17.0,21867.0,18.292404,77.742717,1.0,0.235294
Condado-Campiña,13926.0,27.0,109.0,156231.0,17.282102,69.768484,4.0,0.247706
Villablanca,284.0,3.0,12.0,2885.0,103.986135,415.944541,0.0,0.250000
Bonares,454.0,1.0,4.0,6060.0,16.501650,66.006601,0.0,0.250000
Cortegana,291.0,1.0,4.0,4602.0,21.729683,86.918731,0.0,0.250000
Palma del Condado (La),1414.0,1.0,4.0,10801.0,9.258402,37.033608,0.0,0.250000
Punta Umbría,1255.0,4.0,13.0,15355.0,26.050147,84.662976,0.0,0.307692
Valverde del Camino,832.0,1.0,3.0,12750.0,7.843137,23.529412,0.0,0.333333
